In [ ]:
import os,math
import rasterio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
from glob import glob
from netCDF4 import Dataset
from functools import partial
from pyproj import CRS
from pyproj import Transformer
from shapely.geometry import Point, mapping
from shapely.ops import transform
from scipy.optimize import curve_fit
from sympy import symbols, solve

In [ ]:
## input params
start_yr = 2015
end_yr = 2021
ls_city = ['Atlanta','Austin','Charlotte','Cincinnati','Columbus','Dallas','Houston','Indianapolis','KC','Louisville',
           'Memphis','Miami','Minneapolis','Nashville','NYC','OKC','Omaha','Orlando','Phoenix','Pittsburgh',
           'Richmond','SanAntonio','StLouis','Tucson','Philadelphia','DC','Baltimore']
ls_lon = [-84.3880,-97.7431,-80.8431,-84.5120,-82.9988,-96.7970,-95.3698,-86.1581,-94.5786,-85.7585,-90.0490,
       -80.1918,-93.2650,-86.7816,-74.0060,-97.5164,-95.9345,-81.3789,-112.0740,-79.9959,-77.4360,-98.4936,-90.1994,
          -110.9747,-75.1652,-77.0369,-76.6122]
ls_lat = [33.7490,30.2672,35.2271,39.1031,39.9612,32.7767,29.7604,39.7684,39.0997,38.2527,35.1495,
       25.7617,44.9778,36.1627,40.7128,35.4676,41.2565,28.5384,33.4484,40.4406,37.5407,29.4241,38.6270,32.2226,39.9526,
         38.9072,39.2904]
buf_dis = 60 # radius of the round buffer, in km
ang_range = 45 # degree coverage for each direction

In [ ]:
def geodesic_point_buffer(lat, lon, km):
    # Azimuthal equidistant projection
    aeqd_proj = f"+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0"
    transformer = Transformer.from_crs("EPSG:4326", aeqd_proj, always_xy=True)
    point = Point(lon, lat)
    point_aeqd = transform(transformer.transform, point)
    circle_aeqd = point_aeqd.buffer(km * 1000)
    return mapping(transform(partial(transformer.transform, direction="INVERSE"), circle_aeqd))

def rev_drct(x):
    drct = ['E','NE','N','NW','W','SW','S','SE']
    idx = drct.index(x)
    return drct[(idx+4)%len(drct)]

def fsigmoid(x, L, x0, k, b):
    return L/(1+np.exp(-k*(x-x0)))+b

def drct_px(city,lon,lat):
    
    if not os.path.exists('./Modis/Results'):
        os.mkdir('./Modis/Results')    
    
    file_land = glob('./Modis/*.tif')
    file_land = file_land[0]
    year = file_land.split('_')[-2][3:7]

    ras = rxr.open_rasterio(glob('./Modis/*.tif')[0])
    buffer = geodesic_point_buffer(lat,lon,buf_dis) # create buffer
    raw_buffer = ras.rio.set_crs('WGS84').rio.clip([buffer])
    raw_buffer = raw_buffer[0]
    raw_buffer = raw_buffer.where(raw_buffer < 255)
    px_sum = raw_buffer.count().values
    px_imp = raw_buffer.where(raw_buffer == 13).count().values
    frac_imp = px_imp/px_sum
    frac_imp = round(frac_imp,4)

    center_x = int(np.floor(raw_buffer.x.size/2))
    center_y = int(np.floor(raw_buffer.y.size/2))
    xmat = np.array([[x-raw_buffer.x[center_x-1].values for x in raw_buffer.x.values]]*len(raw_buffer.y.values))
    ymat = np.array([[y-raw_buffer.y[center_y-1].values for y in raw_buffer.y.values]]*len(raw_buffer.x.values)).T
    arr_ang = np.rad2deg(np.arctan2(ymat,xmat))
    arr_ang = np.array(arr_ang<0)*360+arr_ang
    
    ang_buffer = xr.DataArray(
            data = arr_ang,
            dims = ['y','x'], # control the x,y layout when plotting
            coords = dict(
            x = (['x'],raw_buffer.x.values),
            y = (['y'],raw_buffer.y.values),
            )
        )

    temp_all = [city]
    temp_imp = [city]
    wind_val = [0,45,90,135,180,225,270,315]
    for i in wind_val:
        bound_low = i-ang_range/2
        bound_high = i+ang_range/2
    
        if bound_low>=0 and bound_high<360:
            masked = raw_buffer.where((ang_buffer>=bound_low)&(ang_buffer<bound_high))
        if bound_low<0 and bound_high>0:
            masked = raw_buffer.where((ang_buffer>=bound_low+360)|(ang_buffer<bound_high))
        
        px_sum = masked.count().values
        px_imp = masked.where(masked == 13).count().values
        #frac_imp = px_imp/px_sum
        #frac_imp = round(frac_imp,4)
        temp_all.append(px_sum)
        temp_imp.append(px_imp)
    temp_all = [temp_all]
    temp_imp = [temp_imp]
    return temp_all,temp_imp,[[city,frac_imp]]

In [ ]:
os.chdir('/glade/scratch/USERNAME/')
res_px_all = pd.DataFrame()
res_px_imp = pd.DataFrame()
imp_domain = pd.DataFrame()
for city,lon,lat in zip(ls_city,ls_lon,ls_lat):
    res_px_all = res_px_all.append(drct_px(city,lon,lat)[0],ignore_index=True)
    res_px_imp = res_px_imp.append(drct_px(city,lon,lat)[1],ignore_index=True)
    imp_domain = imp_domain.append(drct_px(city,lon,lat)[2],ignore_index=True)
col_name = ['City','E','NE','N','NW','W','SW','S','SE']
res_px_all.columns = col_name
res_px_imp.columns = col_name
imp_domain.columns = ['City','imp_fraction']

res_px_all.to_csv('./Modis/Results/Drct_all_px'+str(buf_dis)+'.csv',sep = ' ')
res_px_imp.to_csv('./Modis/Results/Drct_imp_px'+str(buf_dis)+'.csv',sep = ' ')
imp_domain.to_csv('./Modis/Results/Domain_imp'+str(buf_dis)+'.csv',sep = ' ')

In [ ]:
# generate total precipitation at top wind directions
buf_dis = 60

domain_up = []
domain_dn = []
domain_sum = []
t1_up = [] # store upwind precip at T1 direction
t1_dn = []
t2_up = []
t2_dn = []
top1_drct = []
top2_drct = []
drct_t1_frac = []
drct_t2_frac = []

for city in ls_city:
    dat_wind = pd.DataFrame()#columns=['time','direction','speed']
    dat_domain = pd.DataFrame()
    dat_precip_up = pd.DataFrame()
    dat_precip_dn = pd.DataFrame()
    for year in range(start_yr,end_yr+1):
        os.chdir('/glade/scratch/USERNAME/MRMS_PrecipData/'+str(year))
        filename_wind = 'wind_drct_'+city+'.txt'
        filename_upwind = 'Results/upwind_'+city+str(buf_dis)+'.txt'
        filename_downwind = 'Results/downwind_'+city+str(buf_dis)+'.txt'
        filename_domain = 'Results/domain stat/'+city+str(year)+'_'+str(buf_dis)+'.txt' # don't use the file with no buf_dis
        
        wind = pd.read_csv(filename_wind,sep = ' ',header = None)
        dat_wind = pd.concat([dat_wind,wind],ignore_index=True)
        
        precip_up = pd.read_csv(filename_upwind, sep = ' ')
        dat_precip_up = pd.concat([dat_precip_up,precip_up],ignore_index=True)
        precip_dn = pd.read_csv(filename_downwind, sep = ' ')
        dat_precip_dn = pd.concat([dat_precip_dn,precip_dn],ignore_index=True)
        
        domain_yr = pd.read_csv(filename_domain, sep = ' ')
        dat_domain = pd.concat([dat_domain,domain_yr],ignore_index=True)
        
    dat_wind.columns = ['time','direction','speed']    
    # determine prevailing (top2) wind direction
    #dat_wind['zone'] = dat_wind['direction'].map(wnd_dmnt)
    dat_wind['zone'] = pd.cut(x=dat_wind['direction'],bins=[0]+list(np.arange(22.5,338,45))+[360],right=False,
      labels=['E','NE','N','NW','W','SW','S','SE','E'],ordered=False)
    zone_count = dat_wind['zone'].value_counts()
    #print(zone_count.values)
    zone_max = [zone_count.index[0], zone_count.index[1]]
    # calculate fraction
    drct_t1_frac.append(zone_count.values[0]/sum(zone_count.values))
    drct_t2_frac.append((zone_count.values[0]+zone_count.values[1])/sum(zone_count.values))
        
    colnames = ["time","mean","median","min","max","std","sum","percentile(25)","percentile(75)","percentile(10)",
            "percentile(20)","percentile(30)","percentile(40)","xgrids(90)","xgrids(95)"]
    dat_precip_up.columns = colnames
    dat_precip_up['region'] = 'upwind'
    dat_precip_dn.columns = colnames
    dat_precip_dn['region'] = 'downwind'
    
    no_dup_up = pd.concat([dat_wind.time,dat_precip_up.time]).drop_duplicates(keep=False)
    dat_wind = dat_wind.drop(no_dup_up.index.values)
    res_up = dat_wind.merge(dat_precip_up,how='outer',on='time')
    res_dn = dat_wind.merge(dat_precip_dn,how='outer',on='time')
    
    # apply filter
    res_up = res_up[res_up['mean']>=0.1]
    res_dn = res_dn[res_dn['mean']>=0.1]
    intersect = set(res_up['time'])&set(res_dn['time'])
    #print(len(intersect))
    res_up = res_up[res_up['time'].isin(intersect)]
    res_dn = res_dn[res_dn['time'].isin(intersect)]
    #dat_domain = dat_domain[dat_domain['time'].isin(intersect)]
    domain_up.append(round(res_up['mean'].sum(),4))
    domain_dn.append(round(res_dn['mean'].sum(),4))
    domain_sum.append(round(dat_domain['domain_avg'].sum(),4))
    
    # select by direction
    res_up_t1 = res_up[res_up['zone']==zone_max[0]]
    res_dn_t1 = res_dn[res_dn['zone']==zone_max[0]]
    res_up_t2 = res_up[(res_up['zone']==zone_max[0])|(res_up['zone']==zone_max[1])]
    res_dn_t2 = res_dn[(res_dn['zone']==zone_max[0])|(res_dn['zone']==zone_max[1])]
    top1_drct.append(zone_max[0])
    top2_drct.append(zone_max[0]+'+'+zone_max[1])
    
    t1_up.append(round(res_up_t1['mean'].sum(),4))
    t1_dn.append(round(res_dn_t1['mean'].sum(),4))
    t2_up.append(round(res_up_t2['mean'].sum(),4))
    t2_dn.append(round(res_dn_t2['mean'].sum(),4))

res_domain = pd.DataFrame(
            {'City':ls_city,
             'Upwind_Sum':domain_up,
             'Downwind_Sum':domain_dn,
             'Domain_Sum':domain_sum
            }
            )
res_t1 = pd.DataFrame(
            {'City':ls_city,
             'Upwind_Sum':t1_up,
             'Downwind_Sum':t1_dn,
             'Domain_Sum':domain_sum,
             'Top 1 drct':top1_drct,
             'Fraction':[round(x,4) for x in drct_t1_frac]
            }
            )
res_t2 = pd.DataFrame(
            {'City':ls_city,
             'Upwind_Sum':t2_up,
             'Downwind_Sum':t2_dn,
             'Domain_Sum':domain_sum,
             'Top 2 drct':top2_drct,
             'Fraction':[round(x,4) for x in drct_t2_frac]
            }
            )
os.chdir('/glade/scratch/USERNAME/MRMS_PrecipData/')
res_domain.to_csv('AllDrct_SumPrecip'+str(buf_dis)+'.csv',sep = ' ')
res_t1.to_csv('T1Drct_SumPrecip'+str(buf_dis)+'.csv',sep = ' ')
res_t2.to_csv('T2Drct_SumPrecip'+str(buf_dis)+'.csv',sep = ' ')

# plot the relation betwen Imp and Downwind enhancement

## 0. Test 20km imperviousness in upwind vs. 60km DEI in downwind

In [ ]:
# all directions
buf_imp = 20
buf_DEI = 60
from adjustText import adjust_text

os.chdir('/glade/scratch/USERNAME')
imp_domain = pd.read_csv('./Modis/Results/Domain_imp'+str(buf_imp)+'.csv',sep = ' ')
precip_domain = pd.read_csv('./MRMS_PrecipData/AllDrct_SumPrecip'+str(buf_DEI)+'.csv',sep = ' ')
DEI_domain = (precip_domain['Downwind_Sum']-precip_domain['Upwind_Sum']) \
/(precip_domain['Downwind_Sum']+precip_domain['Upwind_Sum'])
Imp = imp_domain['imp_fraction'].to_list()
#print(imp_domain)
city = imp_domain['City'].to_list()

####
p_val = pd.read_csv('./MRMS_PrecipData/AllDrct_SigLevel'+str(buf_DEI)+'.csv',sep = ' ')
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
Imp_filt = [Imp[i] for i in Sig_idx]
DEI_filt = DEI_domain[Sig_idx]

plt.style.use('seaborn')
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(Imp,DEI_domain)
ax.scatter(Imp_filt,DEI_filt,color='red')
texts = [ax.text(Imp[i],DEI_domain[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)

ax.set_xlabel('Impervious Fraction',size=18)
ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)
ax.set_title('All Wind Directions',size = 20)

In [ ]:
## Top 1 direction

drct_px = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_imp)+'.csv',sep = ' ')
drct_imp = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_imp)+'.csv',sep = ' ')
precip_t1 = pd.read_csv('./MRMS_PrecipData/T1Drct_SumPrecip'+str(buf_DEI)+'.csv',sep = ' ')

T1_drct_up = precip_t1['Top 1 drct'].to_list()
T1_drct_dn = [rev_drct(i) for i in T1_drct_up]

T1drct_px_up = []
T1drct_imp_up = []
for i,j in enumerate(T1_drct_up):
    T1drct_px_up.append(drct_px.iloc[i][j])
    T1drct_imp_up.append(drct_imp.iloc[i][j])


T1drct_px_dn = []
T1drct_imp_dn = []
for i,j in enumerate(T1_drct_dn):
    T1drct_px_dn.append(drct_px.iloc[i][j])
    T1drct_imp_dn.append(drct_imp.iloc[i][j])

frac_imp_up = [i/j for i,j in zip(T1drct_imp_up,T1drct_px_up)]
frac_imp_up = [round(i,4) for i in frac_imp_up]    

frac_imp_dn = [i/j for i,j in zip(T1drct_imp_dn,T1drct_px_dn)]
frac_imp_dn = [round(i,4) for i in frac_imp_dn]
#print(frac_imp)

DEI_t1 = (precip_t1['Downwind_Sum']-precip_t1['Upwind_Sum']) \
/(precip_t1['Downwind_Sum']+precip_t1['Upwind_Sum'])

####
p_val = pd.read_csv('./MRMS_PrecipData/T1Drct_SigLevel'+str(buf_DEI)+'.csv',sep = ' ')
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
imp_up_filt = [frac_imp_up[i] for i in Sig_idx]
DEI_t1_filt = DEI_t1[Sig_idx]

city = precip_t1['City'].to_list()

plt.style.use('seaborn')
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(frac_imp_up,DEI_t1)  #frac_imp_dn
ax.scatter(imp_up_filt,DEI_t1_filt,color='red')
texts = [ax.text(frac_imp_up[i],DEI_t1[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)
#for i,txt in enumerate(city):
#    ax.annotate(txt,(frac_imp_up[i],DEI_domain[i]))
ax.set_xlabel('Upwind Impervious Fraction',size=18)
ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)
ax.set_title('Top1 Prevailing Direction',size = 20)

In [ ]:
## Top 2 directions

drct_px = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_imp)+'.csv',sep = ' ')
drct_imp = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_imp)+'.csv',sep = ' ')
precip_t2 = pd.read_csv('./MRMS_PrecipData/T2Drct_SumPrecip'+str(buf_DEI)+'.csv',sep = ' ')
T2_drct_up = precip_t2['Top 2 drct'].to_list()

T2drct_px_up = []
T2drct_imp_up = []
for i,j in enumerate(T2_drct_up):
    #temp = []
    drcts = j.split('+')
    T2drct_px_up.append(drct_px.iloc[i][drcts[0]]+drct_px.iloc[i][drcts[1]])
    T2drct_imp_up.append(drct_imp.iloc[i][drcts[0]]+drct_imp.iloc[i][drcts[1]])
    
frac_imp_up = [i/j for i,j in zip(T2drct_imp_up,T2drct_px_up)]
frac_imp_up = [round(i,4) for i in frac_imp_up]

T2drct_px_dn = []
T2drct_imp_dn = []
for i,j in enumerate(T2_drct_up):
    #temp = []
    drcts = j.split('+')
    T2drct_px_dn.append(drct_px.iloc[i][rev_drct(drcts[0])]+drct_px.iloc[i][rev_drct(drcts[1])])
    T2drct_imp_dn.append(drct_imp.iloc[i][rev_drct(drcts[0])]+drct_imp.iloc[i][rev_drct(drcts[1])])
    
frac_imp_dn = [i/j for i,j in zip(T2drct_imp_dn,T2drct_px_dn)]
frac_imp_dn = [round(i,4) for i in frac_imp_dn]
#print(frac_imp)

DEI_t2 = (precip_t2['Downwind_Sum']-precip_t2['Upwind_Sum']) \
/(precip_t2['Downwind_Sum']+precip_t2['Upwind_Sum'])

####
p_val = pd.read_csv('./MRMS_PrecipData/T2Drct_SigLevel'+str(buf_DEI)+'.csv',sep = ' ')
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
imp_up_filt = [frac_imp_up[i] for i in Sig_idx]
DEI_t2_filt = DEI_t2[Sig_idx]

city = precip_t2['City'].to_list()

plt.style.use('seaborn')
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(frac_imp_up,DEI_t2) #
ax.scatter(imp_up_filt,DEI_t2_filt,color='red')
texts = [ax.text(frac_imp_up[i],DEI_t2[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)
ax.set_xlabel('Upwind Impervious Fraction',size=18)
ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)
ax.set_title('Top2 Prevailing Directions',size = 20)

## 1. All directions

In [ ]:
buf_dis = 20

from adjustText import adjust_text
os.chdir('/glade/scratch/USERNAME')
imp_domain = pd.read_csv('./Modis/Results/Domain_imp'+str(buf_dis)+'.csv',sep = ' ')
precip_domain = pd.read_csv('./MRMS_PrecipData/AllDrct_SumPrecip'+str(buf_dis)+'.csv',sep = ' ')
DEI_domain = (precip_domain['Downwind_Sum']-precip_domain['Upwind_Sum']) \
/(precip_domain['Downwind_Sum']+precip_domain['Upwind_Sum'])
Imp = imp_domain['imp_fraction'].to_list()
#print(imp_domain)
city = imp_domain['City'].to_list()

####
p_val = pd.read_csv('./MRMS_PrecipData/AllDrct_SigLevel'+str(buf_dis)+'.csv',sep = ' ')
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
Imp_filt = [Imp[i] for i in Sig_idx]
DEI_filt = DEI_domain[Sig_idx]

plt.style.use('seaborn')
#plt.figure(figsize=(10,7))
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(Imp,DEI_domain)
ax.scatter(Imp_filt,DEI_filt,color='red')
texts = [ax.text(Imp[i],DEI_domain[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)
#for i,txt in enumerate(city):
#    ax.annotate(txt,(Imp[i]*(1+0.01),DEI_domain[i]*(1+0.01)))
ax.set_xlabel('Impervious Fraction',size=18)
ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)
ax.set_title('All Wind Directions',size = 20)


## 2. Top1 Direction

In [ ]:
buf_dis = 20
from adjustText import adjust_text
os.chdir('/glade/scratch/USERNAME')

drct_px = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis)+'.csv',sep = ' ')
drct_imp = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis)+'.csv',sep = ' ')
precip_t1 = pd.read_csv('./MRMS_PrecipData/T1Drct_SumPrecip'+str(buf_dis)+'.csv',sep = ' ')

T1_drct_up = precip_t1['Top 1 drct'].to_list()
T1_drct_dn = [rev_drct(i) for i in T1_drct_up]

T1drct_px_up = []
T1drct_imp_up = []
for i,j in enumerate(T1_drct_up):
    T1drct_px_up.append(drct_px.iloc[i][j])
    T1drct_imp_up.append(drct_imp.iloc[i][j])


T1drct_px_dn = []
T1drct_imp_dn = []
for i,j in enumerate(T1_drct_dn):
    T1drct_px_dn.append(drct_px.iloc[i][j])
    T1drct_imp_dn.append(drct_imp.iloc[i][j])

frac_imp_up = [i/j for i,j in zip(T1drct_imp_up,T1drct_px_up)]
frac_imp_up = [round(i,4) for i in frac_imp_up]    

frac_imp_dn = [i/j for i,j in zip(T1drct_imp_dn,T1drct_px_dn)]
frac_imp_dn = [round(i,4) for i in frac_imp_dn]
#print(frac_imp)

DEI_t1 = (precip_t1['Downwind_Sum']-precip_t1['Upwind_Sum']) \
/(precip_t1['Downwind_Sum']+precip_t1['Upwind_Sum'])

####
p_val = pd.read_csv('./MRMS_PrecipData/T1Drct_SigLevel'+str(buf_dis)+'.csv',sep = ' ')
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
imp_up_filt = [frac_imp_up[i] for i in Sig_idx]
DEI_t1_filt = DEI_t1[Sig_idx]

city = precip_t1['City'].to_list()

plt.style.use('seaborn')
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(frac_imp_up,DEI_t1)  #frac_imp_dn
ax.scatter(imp_up_filt,DEI_t1_filt,color='red')
texts = [ax.text(frac_imp_up[i],DEI_t1[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)
#for i,txt in enumerate(city):
#    ax.annotate(txt,(frac_imp_up[i],DEI_domain[i]))
ax.set_xlabel('Upwind Impervious Fraction',size=18)
ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)
ax.set_title('Top1 Prevailing Direction',size = 20)

In [ ]:
T1drct_px_up = []
T1drct_imp_up = []
for i,j in enumerate(T1_drct_up):
    T1drct_px_up.append(drct_px.iloc[i][j])
    T1drct_imp_up.append(drct_imp.iloc[i][j])

frac_imp_up = [i/j for i,j in zip(T1drct_imp_up,T1drct_px_up)]
frac_imp_up = [round(i,4) for i in frac_imp_up]

diff_frac = [(i-j)/(i+j) for i,j in zip(frac_imp_dn,frac_imp_up)]

####
p_val = pd.read_csv('./MRMS_PrecipData/T1Drct_SigLevel'+str(buf_dis)+'.csv',sep = ' ')
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
diff_frac_filt = [diff_frac[i] for i in Sig_idx]
DEI_t1_filt = DEI_t1[Sig_idx]

plt.style.use('seaborn')
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(diff_frac,DEI_t1)
ax.scatter(diff_frac_filt,DEI_t1_filt,color='red')
texts = [ax.text(diff_frac[i],DEI_t1[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)
#for i,txt in enumerate(city):
#    ax.annotate(txt,(Imp[i]*(1+0.01),DEI_domain[i]*(1+0.01)))
ax.set_xlabel('Normalized Difference Imperviousness: $Imp_{down}-Imp_{up}/(Imp_{down}+Imp_{up})$',size=18)
ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)

## 3. Top 2 Directions

In [ ]:
buf_dis = 60
from adjustText import adjust_text
os.chdir('/glade/scratch/USERNAME')

drct_px = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis)+'.csv',sep = ' ')
drct_imp = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis)+'.csv',sep = ' ')
precip_t2 = pd.read_csv('./MRMS_PrecipData/T2Drct_SumPrecip'+str(buf_dis)+'.csv',sep = ' ')
T2_drct_up = precip_t2['Top 2 drct'].to_list()

T2drct_px_up = []
T2drct_imp_up = []
for i,j in enumerate(T2_drct_up):
    #temp = []
    drcts = j.split('+')
    T2drct_px_up.append(drct_px.iloc[i][drcts[0]]+drct_px.iloc[i][drcts[1]])
    T2drct_imp_up.append(drct_imp.iloc[i][drcts[0]]+drct_imp.iloc[i][drcts[1]])
    
frac_imp_up = [i/j for i,j in zip(T2drct_imp_up,T2drct_px_up)]
frac_imp_up = [round(i,4) for i in frac_imp_up]

T2drct_px_dn = []
T2drct_imp_dn = []
for i,j in enumerate(T2_drct_up):
    #temp = []
    drcts = j.split('+')
    T2drct_px_dn.append(drct_px.iloc[i][rev_drct(drcts[0])]+drct_px.iloc[i][rev_drct(drcts[1])])
    T2drct_imp_dn.append(drct_imp.iloc[i][rev_drct(drcts[0])]+drct_imp.iloc[i][rev_drct(drcts[1])])
    
frac_imp_dn = [i/j for i,j in zip(T2drct_imp_dn,T2drct_px_dn)]
frac_imp_dn = [round(i,4) for i in frac_imp_dn]
#print(frac_imp)

DEI_t2 = (precip_t2['Downwind_Sum']-precip_t2['Upwind_Sum']) \
/(precip_t2['Downwind_Sum']+precip_t2['Upwind_Sum'])

####
p_val = pd.read_csv('./MRMS_PrecipData/T2Drct_SigLevel'+str(buf_dis)+'.csv',sep = ' ')
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
imp_up_filt = [frac_imp_up[i] for i in Sig_idx]
DEI_t2_filt = DEI_t2[Sig_idx]

city = precip_t2['City'].to_list()

plt.style.use('seaborn')
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(frac_imp_up,DEI_t2) #
ax.scatter(imp_up_filt,DEI_t2_filt,color='red')
texts = [ax.text(frac_imp_up[i],DEI_t2[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)
ax.set_xlabel('Upwind Impervious Fraction',size=18)
ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)
ax.set_title('Top2 Prevailing Directions',size = 20)

In [ ]:
T2drct_px_up = []
T2drct_imp_up = []
for i,j in enumerate(T2_drct_up):
    #temp = []
    drcts = j.split('+')
    T2drct_px_up.append(drct_px.iloc[i][drcts[0]]+drct_px.iloc[i][drcts[1]])
    T2drct_imp_up.append(drct_imp.iloc[i][drcts[0]]+drct_imp.iloc[i][drcts[1]])
    
frac_imp_up = [i/j for i,j in zip(T2drct_imp_up,T2drct_px_up)]
frac_imp_up = [round(i,4) for i in frac_imp_up]

T2drct_px_dn = []
T2drct_imp_dn = []
for i,j in enumerate(T2_drct_up):
    #temp = []
    drcts = j.split('+')
    T2drct_px_dn.append(drct_px.iloc[i][rev_drct(drcts[0])]+drct_px.iloc[i][rev_drct(drcts[1])])
    T2drct_imp_dn.append(drct_imp.iloc[i][rev_drct(drcts[0])]+drct_imp.iloc[i][rev_drct(drcts[1])])
    
frac_imp_dn = [i/j for i,j in zip(T2drct_imp_dn,T2drct_px_dn)]
frac_imp_dn = [round(i,4) for i in frac_imp_dn]

diff_frac = [(i-j) for i,j in zip(frac_imp_dn,frac_imp_up)]

####
p_val = pd.read_csv('./MRMS_PrecipData/T2Drct_SigLevel'+str(buf_dis)+'.csv',sep = ' ')
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
diff_frac_filt = [diff_frac[i] for i in Sig_idx]
DEI_t2_filt = DEI_t2[Sig_idx]

plt.style.use('seaborn')
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(diff_frac,DEI_t2)
ax.scatter(diff_frac_filt,DEI_t2_filt,color='red')
texts = [ax.text(diff_frac[i],DEI_t2[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)
#for i,txt in enumerate(city):
#    ax.annotate(txt,(Imp[i]*(1+0.01),DEI_domain[i]*(1+0.01)))
ax.set_xlabel('Normalized Difference Imperviousness: $Imp_{down}-Imp_{up}/(Imp_{down}+Imp_{up})$',size=18)
ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)

## 4. Upwind imp gradient vs. DEI

In [ ]:
############# Top 2 directions #############
############# 60km (20-60) vs 20km (0-20)##################

buf_dis1 = 60
buf_dis2 = 20
from adjustText import adjust_text
os.chdir('/glade/scratch/USERNAME')

drct_px_60 = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis1)+'.csv',sep = ' ') # number of all pixels
drct_imp_60 = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis1)+'.csv',sep = ' ') # number of impervious pixels
drct_px_20 = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis2)+'.csv',sep = ' ') # number of all pixels
drct_imp_20 = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis2)+'.csv',sep = ' ') # number of impervious pixels

precip_t2 = pd.read_csv('./MRMS_PrecipData/T2Drct_SumPrecip'+str(buf_dis1)+'.csv',sep = ' ') # use 60km DEI
T2_drct_up = precip_t2['Top 2 drct'].to_list()

T2drct_px60_up = []
T2drct_imp60_up = []
T2drct_px20_up = []
T2drct_imp20_up = []
for i,j in enumerate(T2_drct_up):
    drcts = j.split('+')
    T2drct_px60_up.append(drct_px_60.iloc[i][drcts[0]]+drct_px_60.iloc[i][drcts[1]])
    T2drct_imp60_up.append(drct_imp_60.iloc[i][drcts[0]]+drct_imp_60.iloc[i][drcts[1]])
    T2drct_px20_up.append(drct_px_20.iloc[i][drcts[0]]+drct_px_20.iloc[i][drcts[1]])
    T2drct_imp20_up.append(drct_imp_20.iloc[i][drcts[0]]+drct_imp_20.iloc[i][drcts[1]])

T2drct_r2060_px = [i-j for i,j in zip(T2drct_px60_up,T2drct_px20_up)] ## number of total pixels in the ring between 20km and 60km
T2drct_r2060_imp = [i-j for i,j in zip(T2drct_imp60_up,T2drct_imp20_up)]

frac_imp2060_up = [i/j for i,j in zip(T2drct_r2060_imp,T2drct_r2060_px)]
frac_imp2060_up = [round(i,4) for i in frac_imp2060_up]
print(frac_imp2060_up)
frac_imp60_up = [i/j for i,j in zip(T2drct_imp60_up,T2drct_px60_up)]
frac_imp60_up = [round(i,4) for i in frac_imp60_up]
print(frac_imp60_up)

frac_imp20_up = [i/j for i,j in zip(T2drct_imp20_up,T2drct_px20_up)]
frac_imp20_up = [round(i,4) for i in frac_imp20_up]


gradient = [i/j for i,j in zip(frac_imp20_up,frac_imp60_up)]
gradient = [round(i,2) for i in gradient]


DEI_t2 = 2*(precip_t2['Downwind_Sum']-precip_t2['Upwind_Sum']) \
/(precip_t2['Downwind_Sum']+precip_t2['Upwind_Sum'])

####
p_val = pd.read_csv('./MRMS_PrecipData/T2Drct_SigLevel'+str(buf_dis1)+'.csv',sep = ' ') # use the 60km significance results
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
gradient_filt = [gradient[i] for i in Sig_idx]
DEI_t2_filt = DEI_t2[Sig_idx]

city = precip_t2['City'].to_list()
print(np.corrcoef(gradient_filt,DEI_t2_filt))

plt.style.use('seaborn')
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(gradient,DEI_t2) #
ax.scatter(gradient_filt,DEI_t2_filt,color='red')
texts = [ax.text(gradient[i],DEI_t2[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)
ax.set_xlabel('Upwind Impervious Ratio (20km/60km)',size=18)
ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)
ax.set_title('Top2 Prevailing Directions',size = 20)

In [ ]:
############# Top 2 directions #############
############# 40km & 20km ##################
buf_dis1 = 40
buf_dis2 = 20
from adjustText import adjust_text
os.chdir('/glade/scratch/USERNAME')

drct_px_40 = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis1)+'.csv',sep = ' ') # number of all pixels
drct_imp_40 = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis1)+'.csv',sep = ' ') # number of impervious pixels
drct_px_20 = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis2)+'.csv',sep = ' ') # number of all pixels
drct_imp_20 = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis2)+'.csv',sep = ' ') # number of impervious pixels

precip_t2 = pd.read_csv('./MRMS_PrecipData/T2Drct_SumPrecip'+str(buf_dis1)+'.csv',sep = ' ') # use 60km DEI
T2_drct_up = precip_t2['Top 2 drct'].to_list()

T2drct_px40_up = []
T2drct_imp40_up = []
T2drct_px20_up = []
T2drct_imp20_up = []
for i,j in enumerate(T2_drct_up):
    drcts = j.split('+')
    T2drct_px40_up.append(drct_px_40.iloc[i][drcts[0]]+drct_px_40.iloc[i][drcts[1]])
    T2drct_imp40_up.append(drct_imp_40.iloc[i][drcts[0]]+drct_imp_40.iloc[i][drcts[1]])
    T2drct_px20_up.append(drct_px_20.iloc[i][drcts[0]]+drct_px_20.iloc[i][drcts[1]])
    T2drct_imp20_up.append(drct_imp_20.iloc[i][drcts[0]]+drct_imp_20.iloc[i][drcts[1]])
    
frac_imp40_up = [i/j for i,j in zip(T2drct_imp40_up,T2drct_px40_up)]
frac_imp40_up = [round(i,4) for i in frac_imp40_up]

frac_imp20_up = [i/j for i,j in zip(T2drct_imp20_up,T2drct_px20_up)]
frac_imp20_up = [round(i,4) for i in frac_imp20_up]


gradient = [i/j for i,j in zip(frac_imp40_up,frac_imp20_up)]
gradient = [round(i,2) for i in gradient]


DEI_t2 = (precip_t2['Downwind_Sum']-precip_t2['Upwind_Sum']) \
/(precip_t2['Downwind_Sum']+precip_t2['Upwind_Sum'])

####
p_val = pd.read_csv('./MRMS_PrecipData/T2Drct_SigLevel'+str(buf_dis1)+'.csv',sep = ' ') # use the 60km significance results
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()

gradient_filt = [gradient[i] for i in Sig_idx]
DEI_t2_filt = DEI_t2[Sig_idx]

city = precip_t2['City'].to_list()
print(np.corrcoef(gradient_filt,DEI_t2_filt))

plt.style.use('seaborn')
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(gradient,DEI_t2) #
ax.scatter(gradient_filt,DEI_t2_filt,color='red')
texts = [ax.text(gradient[i],DEI_t2[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)
ax.set_xlabel('Upwind Impervious Gradient (40km/20km)',size=18)
ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)
ax.set_title('Top2 Prevailing Directions',size = 20)

## Imp gradient at 3km radius increment 

In [ ]:
## input params
start_yr = 2015
end_yr = 2021
ls_city = ['Atlanta','Austin','Charlotte','Cincinnati','Columbus','Dallas','Houston','Indianapolis','KC','Louisville',
           'Memphis','Miami','Minneapolis','Nashville','NYC','OKC','Omaha','Orlando','Phoenix','Pittsburgh',
           'Richmond','SanAntonio','StLouis','Tucson','Philadelphia','DC','Baltimore']
ls_lon = [-84.3880,-97.7431,-80.8431,-84.5120,-82.9988,-96.7970,-95.3698,-86.1581,-94.5786,-85.7585,-90.0490,
       -80.1918,-93.2650,-86.7816,-74.0060,-97.5164,-95.9345,-81.3789,-112.0740,-79.9959,-77.4360,-98.4936,-90.1994,
          -110.9747,-75.1652,-77.0369,-76.6122]
ls_lat = [33.7490,30.2672,35.2271,39.1031,39.9612,32.7767,29.7604,39.7684,39.0997,38.2527,35.1495,
       25.7617,44.9778,36.1627,40.7128,35.4676,41.2565,28.5384,33.4484,40.4406,37.5407,29.4241,38.6270,32.2226,39.9526,
         38.9072,39.2904]
#buf_dis = 3 # radius of the round buffer, in km
ang_range = 45 # degree coverage for each direction

In [ ]:
os.chdir('/glade/scratch/USERNAME/')
for buf_dis in np.arange(3,63,3):    
    res_px_all = pd.DataFrame()
    res_px_imp = pd.DataFrame()
    imp_domain = pd.DataFrame()
    for city,lon,lat in zip(ls_city,ls_lon,ls_lat):
        res_px_all = res_px_all.append(drct_px(city,lon,lat)[0],ignore_index=True)
        res_px_imp = res_px_imp.append(drct_px(city,lon,lat)[1],ignore_index=True)
        imp_domain = imp_domain.append(drct_px(city,lon,lat)[2],ignore_index=True) # overall fraction of imp (non-directional)
    col_name = ['City','E','NE','N','NW','W','SW','S','SE']
    res_px_all.columns = col_name
    res_px_imp.columns = col_name
    imp_domain.columns = ['City','imp_fraction']

    res_px_all.to_csv('./Modis/Results/Imp gradient/Drct_all_px'+str(buf_dis).zfill(2)+'.csv',sep = ' ')
    res_px_imp.to_csv('./Modis/Results/Imp gradient/Drct_imp_px'+str(buf_dis).zfill(2)+'.csv',sep = ' ')
    imp_domain.to_csv('./Modis/Results/Imp gradient/Domain_imp'+str(buf_dis).zfill(2)+'.csv',sep = ' ')

In [ ]:
## plot gradual imp from -60km to 60km (top1 direction)
os.chdir('/glade/scratch/USERNAME/Modis/Results/Imp gradient/')

file_allpx = glob("Drct_all*")
file_imppx = glob("Drct_imp*")
# list to store prevailing directions
tmp = pd.read_csv('/glade/scratch/zhouyu/MRMS_PrecipData/T1Drct_SumPrecip60.csv',sep=' ')
ls_drct = tmp['Top 1 drct'].to_list()  # follow the order of cities


# 27 cities, 20 radius class
df_imp_rate_t1 = pd.DataFrame()
#px_all = pd.DataFrame()
#px_imp = pd.DataFrame()
for drct,city in zip(ls_drct,ls_city):
    ls_all_up = []
    ls_imp_up = []
    ls_all_dn = []
    ls_imp_dn = []
    for f_all,f_imp in zip(sorted(file_allpx),sorted(file_imppx)):
        df_all = pd.read_csv(f_all,sep = ' ')
        df_imp = pd.read_csv(f_imp,sep = ' ')
        ls_all_up.append(df_all[df_all['City']==city][drct].values[0])
        ls_imp_up.append(df_imp[df_imp['City']==city][drct].values[0])
    for f_all,f_imp in zip(sorted(file_allpx),sorted(file_imppx)):
        df_all = pd.read_csv(f_all,sep = ' ')
        df_imp = pd.read_csv(f_imp,sep = ' ')
        ls_all_dn.append(df_all[df_all['City']==city][rev_drct(drct)].values[0])
        ls_imp_dn.append(df_imp[df_imp['City']==city][rev_drct(drct)].values[0])
    #px_all[city] = ls_all
    imp_rate_up = [ls_imp_up[0]/ls_all_up[0]]
    for k in range(1,len(file_allpx)):
        imp_rate_up.append((ls_imp_up[k]-ls_imp_up[k-1])/(ls_all_up[k]-ls_all_up[k-1]))
    imp_rate_up = imp_rate_up[::-1]
    imp_rate_dn = [ls_imp_dn[0]/ls_all_dn[0]]
    for k in range(1,len(file_allpx)):
        imp_rate_dn.append((ls_imp_dn[k]-ls_imp_dn[k-1])/(ls_all_dn[k]-ls_all_dn[k-1]))
    imp_rate = imp_rate_up+imp_rate_dn
    imp_rate = [round(i,3) for i in imp_rate]
    df_imp_rate_t1[city] = imp_rate
    df_imp_rate_t1[df_imp_rate_t1 < 0] = 0
#print(df_imp_rate)

In [ ]:
x = np.arange(-60,60,3)
with plt.style.context(['science','no-latex']):
    fig,ax = plt.subplots(3,3, sharey='row',figsize=(15,15)) # ,sharex='col'
    for i,city in enumerate(ls_city[18:27]):
        ax[int(np.floor(i/3)),i%3].plot(x,df_imp_rate[city],lw = 2)
        #ax[int(np.floor(i/3)),i%3].set_xlabel('Distance (km)',size=16)
        #ax[int(np.floor(i/3)),i%3].set_ylabel('Imperviousness',size=16)
        ax[int(np.floor(i/3)),i%3].set_title(city,size=16)
    fig.text(0.5,0.08,'Distance (km)',size=24,ha='center')
    fig.text(0.08,0.5,'Imperviousness',size=24,va='center',rotation='vertical')
    #plt.ylabel('Imperviousness',size=16)

In [ ]:
## plot gradual imp from -60km to 60km (top2 directions)
os.chdir('/glade/scratch/USERNAME/Modis/Results/Imp gradient/')

file_allpx = glob("Drct_all*")
file_imppx = glob("Drct_imp*")
# list to store prevailing directions
tmp = pd.read_csv('/glade/scratch/zhouyu/MRMS_PrecipData/T2Drct_SumPrecip60.csv',sep=' ')
ls_drct = tmp['Top 2 drct'].to_list()  # follow the order of cities


# 27 cities, 20 radius class
df_imp_rate_t2 = pd.DataFrame()
for drct,city in zip(ls_drct,ls_city):
    ls_all_up = []
    ls_imp_up = []
    ls_all_dn = []
    ls_imp_dn = []
    d1 = drct.split('+')[0]
    d2 = drct.split('+')[1]
    for f_all,f_imp in zip(sorted(file_allpx),sorted(file_imppx)):
        df_all = pd.read_csv(f_all,sep = ' ')
        df_imp = pd.read_csv(f_imp,sep = ' ')
        ls_all_up.append(df_all[df_all['City']==city][d1].values[0]+df_all[df_all['City']==city][d2].values[0])
        ls_imp_up.append(df_imp[df_imp['City']==city][d1].values[0]+df_imp[df_imp['City']==city][d2].values[0])
    for f_all,f_imp in zip(sorted(file_allpx),sorted(file_imppx)):
        df_all = pd.read_csv(f_all,sep = ' ')
        df_imp = pd.read_csv(f_imp,sep = ' ')
        ls_all_dn.append(df_all[df_all['City']==city][rev_drct(d1)].values[0]+df_all[df_all['City']==city][rev_drct(d2)].values[0])
        ls_imp_dn.append(df_imp[df_imp['City']==city][rev_drct(d1)].values[0]+df_imp[df_imp['City']==city][rev_drct(d2)].values[0])
    #px_all[city] = ls_all
    imp_rate_up = [ls_imp_up[0]/ls_all_up[0]]
    for k in range(1,len(file_allpx)):
        imp_rate_up.append((ls_imp_up[k]-ls_imp_up[k-1])/(ls_all_up[k]-ls_all_up[k-1]))
    imp_rate_up = imp_rate_up[::-1]
    imp_rate_dn = [ls_imp_dn[0]/ls_all_dn[0]]
    for k in range(1,len(file_allpx)):
        imp_rate_dn.append((ls_imp_dn[k]-ls_imp_dn[k-1])/(ls_all_dn[k]-ls_all_dn[k-1]))
    imp_rate = imp_rate_up+imp_rate_dn
    imp_rate = [round(i,3) for i in imp_rate]
    df_imp_rate_t2[city] = imp_rate
    df_imp_rate_t2[df_imp_rate_t2 < 0] = 0
    df_imp_rate_t2[df_imp_rate_t2 > 1] = 1
#print(df_imp_rate_t2)

In [ ]:
x = np.arange(-60,60,3)
with plt.style.context(['science','no-latex']):
    fig,ax = plt.subplots(3,3, sharey='row',figsize=(15,15)) # ,sharex='col'
    for i,city in enumerate(ls_city[9:18]):
        ax[int(np.floor(i/3)),i%3].plot(x,df_imp_rate_t2[city],lw = 2)
        #ax[int(np.floor(i/3)),i%3].set_xlabel('Distance (km)',size=16)
        #ax[int(np.floor(i/3)),i%3].set_ylabel('Imperviousness',size=16)
        ax[int(np.floor(i/3)),i%3].set_title(city,size=16)
    fig.text(0.5,0.08,'Distance (km)',size=24,ha='center')
    fig.text(0.08,0.5,'Imperviousness',size=24,va='center',rotation='vertical')

In [ ]:
## plot t1 and t2 results together
x = np.arange(-60,60,3)
with plt.style.context(['science','no-latex']):
    fig,ax = plt.subplots(3,3, sharey='row',figsize=(15,15)) # ,sharex='col'
    for i,city in enumerate(ls_city[18:27]):
        ax[int(np.floor(i/3)),i%3].plot(x,df_imp_rate_t1[city],lw = 2,label='top 1 direction')
        ax[int(np.floor(i/3)),i%3].plot(x,df_imp_rate_t2[city],lw = 2,label='top 2 directions')
        ax[int(np.floor(i/3)),i%3].set_title(city,size=16)
        ax[int(np.floor(i/3)),i%3].legend(loc='upper left')
    fig.text(0.5,0.08,'Distance (km)',size=24,ha='center')
    fig.text(0.08,0.5,'Imperviousness',size=24,va='center',rotation='vertical')

In [ ]:
## grouping based on upwind imp ratio (20km/60km)
group1 = ['DC','Philadelphia','StLouis','Houston']
group2 = ['Baltimore','Dallas','KC','Charlotte','Orlando','Minneapolis']
group3 = ['Memphis','Austin','Columbus','Cincinnati','Omaha','SanAntonio']

x = np.arange(-60,60,3)
with plt.style.context(['science','no-latex']):
    fig,ax = plt.subplots(1,1,figsize=(8,8)) # ,sharex='col'
    for i,city in enumerate(group3):
        ax.plot(x,df_imp_rate_t2[city],lw = 2,label=city)
        #ax[int(np.floor(i/3)),i%3].plot(x,df_imp_rate_t2[city],lw = 2,label='top 2 directions')
        #ax[int(np.floor(i/3)),i%3].set_title(city,size=16)
        ax.legend(loc='upper left')
    fig.text(0.5,0.04,'Distance (km)',size=24,ha='center')
    fig.text(0.04,0.5,'Imperviousness',size=24,va='center',rotation='vertical')

In [ ]:
group1 = ['DC','Philadelphia','StLouis','Houston']
group2 = ['Baltimore','Dallas','KC','Charlotte','Orlando','Minneapolis']
group3 = ['Memphis','Austin','Columbus','Cincinnati','Omaha','SanAntonio']
group_sig = group1+group2+group3
group_nonsig = ['NYC','Phoenix','Pittsburgh','Atlanta','OKC','Richmond','Miami','Nashville','Tucson','Indianapolis','Louisville']

x = np.arange(-60,60,3)
with plt.style.context(['science','no-latex']):
    fig,ax = plt.subplots(1,1,figsize=(8,8)) # ,sharex='col'
    for i,city in enumerate(group_sig):
        ax.plot(x,df_imp_rate_t2[city],'k-',lw = 2 )
        #ax[int(np.floor(i/3)),i%3].plot(x,df_imp_rate_t2[city],lw = 2,label='top 2 directions')
        #ax[int(np.floor(i/3)),i%3].set_title(city,size=16)
        #ax.legend(loc='upper left')
    for i,city in enumerate(group_nonsig):
        ax.plot(x,df_imp_rate_t2[city],'r--',lw = 2 )
    fig.text(0.5,0.04,'Distance (km)',size=24,ha='center')
    fig.text(0.04,0.5,'Imperviousness',size=24,va='center',rotation='vertical')

In [ ]:
### test cell
print(ls_city)
x = np.arange(-60,60,3)
x_up = [i for i in x if i <= 0]
frac_imp_up = df_imp_rate_t2.iloc[0:len(x_up)]
for city in ls_city:
    ydata = frac_imp_up[city].to_list()
    p0 = [max(ydata), np.median(x_up),1,min(ydata)]
    popt, pcov = curve_fit(fsigmoid, x_up, ydata, p0,maxfev=8000)
    yfit = fsigmoid(x_up,*popt)
    if city != 'NYC':
        idx_urb = next(x for x,val in enumerate(yfit) if val>=0.5)
        print(-x_up[idx_urb])

In [ ]:
########### sigmoidal fitting #############

plt.style.use(['science','no-latex'])
x = np.arange(-60,60,3)
x_up = [i for i in x if i <= 0]
frac_imp_up = df_imp_rate_t2.iloc[0:len(x_up)]

fig,ax = plt.subplots(3,3, sharey='row',figsize=(15,15))
for i,city in enumerate(ls_city[0:9]):
    ydata = frac_imp_up[city].to_list()
    p0 = [max(ydata), np.median(x_up),1,min(ydata)]
    popt, pcov = curve_fit(fsigmoid, x_up, ydata, p0,maxfev=8000)
    popt_ls = popt.tolist()
    #print(type(popt_ls[1]))
    yfit = fsigmoid(np.linspace(-60,0,num = 120),*popt)
    if city != 'NYC':
        #root = fsolve(sigm_solve,[1,1],(popt_ls[0],popt_ls[1],popt_ls[2],popt_ls[3]),maxfev=5000) #,popt_ls[0],popt_ls[1],popt_ls[2],popt_ls[3]
        idx_l,imp_l = next((x,val) for x,val in enumerate(yfit) if val>=0.1)
        idx_h,imp_h = next((x,val) for x,val in enumerate(yfit) if val>=0.9)
        slope = (imp_h-imp_l)/(np.linspace(-60,0,num = 120)[idx_h]-np.linspace(-60,0,num = 120)[idx_l])
        ax[int(np.floor(i/3)),i%3].text(0.05,0.9,'Slope:%.4f ($km^{-1}$)'%slope,
                                    transform=ax[int(np.floor(i/3)),i%3].transAxes)
    
    ax[int(np.floor(i/3)),i%3].set_title(city,size=16)
    if city in group_sig:
        ax[int(np.floor(i/3)),i%3].set_title(city,size=16,color='red')
    ax[int(np.floor(i/3)),i%3].scatter(x_up,frac_imp_up[city])
    ax[int(np.floor(i/3)),i%3].plot(np.linspace(-60,0,num = 60),fsigmoid(np.linspace(-60,0,num = 60),*popt),'r-')
    
    #ax[int(np.floor(i/3)),i%3].legend(loc='upper left')
fig.text(0.5,0.08,'Distance (km)',size=24,ha='center')
fig.text(0.08,0.5,'Imperviousness',size=24,va='center',rotation='vertical')

In [ ]:
### 
#imp_fit = pd.DataFrame()
fig,ax = plt.subplots(1,1,figsize=(8,8)) # ,sharex='col'

x = np.arange(-60,60,3)
x_up = [i for i in x if i <= 0]
frac_imp_up = df_imp_rate_t2.iloc[0:len(x_up)]

for i,city in enumerate(group_sig):
    ydata = frac_imp_up[city].to_list()
    p0 = [max(ydata), np.median(x_up),1,min(ydata)]
    popt, pcov = curve_fit(fsigmoid, x_up, ydata, p0,maxfev=8000)
    yfit = fsigmoid(np.linspace(-60,0,num = 60),*popt)
    ax.plot(np.linspace(-60,0,num = 60),yfit,'r-',lw = 2 )
    
for i,city in enumerate(group_nonsig):
    ydata = frac_imp_up[city].to_list()
    p0 = [max(ydata), np.median(x_up),1,min(ydata)]
    popt, pcov = curve_fit(fsigmoid, x_up, ydata, p0,maxfev=8000)
    yfit = fsigmoid(np.linspace(-60,0,num = 60),*popt)
    ax.plot(np.linspace(-60,0,num = 60),yfit,'k--',lw = 2 )
fig.text(0.5,0.04,'Distance (km)',size=24,ha='center')
fig.text(0.04,0.5,'Imperviousness',size=24,va='center',rotation='vertical')

In [ ]:
##### upwind+downwind sigmoidal fitting #####
def fparab(x, a,b,c):
    return -a*(x-b)**2+c

def fsigmoid(x, L, x0, k, b):
    return L/(1+np.exp(-k*(x-x0)))+b

def fsigmoid2(x, L, x0, k, b):
    return L/(1+np.exp(k*(x-x0)))+b

plt.style.use(['science','no-latex'])

x = np.arange(-60,60,3)
x_up = [i for i in x if i <= 0]
frac_imp_up = df_imp_rate_t2.iloc[0:len(x_up)]
x_dn = [i for i in x if i >= 0]
frac_imp_dn = df_imp_rate_t2.iloc[20:]  # 20 is hard-coded

#frac_imp_up = df_imp_rate_t2.iloc[0:len(x_up)]
fig,ax = plt.subplots(3,3, sharey='row',figsize=(15,15))
for i,city in enumerate(ls_city[18:27]):
    ydata_dn = frac_imp_dn[city].to_list()
    p0_dn = [max(ydata_dn), np.median(x_dn),1,min(ydata_dn)]
    #p0 = [1, np.median(x), max(ydata)]
    popt_dn, pcov_dn = curve_fit(fsigmoid2, x_dn, ydata_dn, p0_dn,maxfev=8000)
    popt_ls_dn = popt_dn.tolist()
    yfit_dn = fsigmoid2(np.linspace(0,60,num = 120),*popt_dn)
    
    ydata_up = frac_imp_up[city].to_list()
    p0_up = [max(ydata_up), np.median(x_up),1,min(ydata_up)]
    #p0 = [1, np.median(x), max(ydata)]
    popt_up, pcov_up = curve_fit(fsigmoid, x_up, ydata_up, p0_up,maxfev=8000)
    popt_ls_up = popt_up.tolist()
    yfit_up = fsigmoid2(np.linspace(0,60,num = 120),*popt_up)
    
    ax[int(np.floor(i/3)),i%3].set_title(city,size=16)
    if city in group_sig:
        ax[int(np.floor(i/3)),i%3].set_title(city,size=16,color='red')
        
    ax[int(np.floor(i/3)),i%3].scatter(x_dn,frac_imp_dn[city])
    ax[int(np.floor(i/3)),i%3].plot(np.linspace(0,60,num = 120),fsigmoid2(np.linspace(0,60,num = 120),*popt_dn),'r-')
    ax[int(np.floor(i/3)),i%3].scatter(x_up,frac_imp_up[city])
    ax[int(np.floor(i/3)),i%3].plot(np.linspace(-60,0,num = 120),fsigmoid(np.linspace(-60,0,num = 120),*popt_up),'r-')
    
    #ax[int(np.floor(i/3)),i%3].legend(loc='upper left')
fig.text(0.5,0.08,'Distance (km)',size=24,ha='center')
fig.text(0.08,0.5,'Imperviousness',size=24,va='center',rotation='vertical')

In [ ]:
#### Gaussian fitting ####
def fGauss(x,a,b,c,d):
    return a*np.exp(-b*(x-c)**2)+d

plt.style.use(['science','no-latex'])

x = np.arange(-60,60,3)
fig,ax = plt.subplots(3,3, sharey='row',figsize=(15,15))
for i,city in enumerate(ls_city[0:9]):
    params,cov = curve_fit(fGauss,x,df_imp_rate_t2[city].to_list(),maxfev = 8000)
    yfit = fGauss(x,*params)
    ax[int(np.floor(i/3)),i%3].scatter(x,df_imp_rate_t2[city])
    #ax[int(np.floor(i/3)),i%3].plot(x,yfit,'r-')
    ax[int(np.floor(i/3)),i%3].plot(np.linspace(-60,60,num = 240),fGauss(np.linspace(-60,60,num = 240),*params),'r-')
    
    ax[int(np.floor(i/3)),i%3].set_xlim(-65,65)
    ax[int(np.floor(i/3)),i%3].set_title(city,size=16)
    if city in group_sig:
        ax[int(np.floor(i/3)),i%3].set_title(city,size=16,color='red')
    ax[int(np.floor(i/3)),i%3].axvline(x=params[2], color='k',ls='--')
fig.text(0.5,0.08,'Distance (km)',size=24,ha='center')
fig.text(0.08,0.5,'Imperviousness',size=24,va='center',rotation='vertical')

In [ ]:
## all slopes
ls_slope = []
x = np.arange(-60,60,3)
x_up = [i for i in x if i <= 0]
frac_imp_up = df_imp_rate_t2.iloc[0:len(x_up)]
for city in ls_city:
    ydata = frac_imp_up[city].to_list()
    p0 = [max(ydata), np.median(x_up),1,min(ydata)]
    popt, pcov = curve_fit(fsigmoid, x_up, ydata, p0,maxfev=8000)
    yfit = fsigmoid(np.linspace(-60,0,num = 120),*popt)
    if city != 'NYC':
        idx_l,imp_l = next((x,val) for x,val in enumerate(yfit) if val>=0.1)
        idx_h,imp_h = next((x,val) for x,val in enumerate(yfit) if val>=0.9)
        slope = (imp_h-imp_l)/(np.linspace(-60,0,num = 120)[idx_h]-np.linspace(-60,0,num = 120)[idx_l])
        ls_slope.append(round(slope,4))
    else:
        ls_slope.append('NAN')
print(ls_slope)

In [ ]:
## city size, upwind imp slope, significance, DEI -- four-element plotting
buf_dis1 = 60
buf_dis2 = 20
from adjustText import adjust_text
os.chdir('/glade/scratch/USERNAME')

plt.style.context(['science','no-latex'])

drct_px_60 = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis1)+'.csv',sep = ' ') # number of all pixels
drct_imp_60 = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis1)+'.csv',sep = ' ') # number of impervious pixels
drct_px_20 = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis2)+'.csv',sep = ' ') # number of all pixels
drct_imp_20 = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis2)+'.csv',sep = ' ') # number of impervious pixels

precip_t2 = pd.read_csv('./MRMS_PrecipData/T2Drct_SumPrecip'+str(buf_dis1)+'.csv',sep = ' ') # use 60km DEI
T2_drct_up = precip_t2['Top 2 drct'].to_list()

DEI_t2 = 2*(precip_t2['Downwind_Sum']-precip_t2['Upwind_Sum']) \
/(precip_t2['Downwind_Sum']+precip_t2['Upwind_Sum'])

####
p_val = pd.read_csv('./MRMS_PrecipData/T2Drct_SigLevel'+str(buf_dis1)+'.csv',sep = ' ') # use the 60km significance results
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
#gradient_filt = [gradient[i] for i in Sig_idx]
DEI_t2_filt = DEI_t2[Sig_idx].to_list()

urb_size = []
## read csv containing city size
dat = pd.read_csv('Modis/df_urban2.csv', sep = ' ')
for city in ls_city:
    urb_size.append(dat[dat['City']==city]['UrbanSize'].values.item())

urb_size = [float(i) for i in urb_size]
print(urb_size)

df = pd.DataFrame()
df['City'] = ls_city
df['DEI'] = DEI_t2
df['UrbanSize'] = urb_size
df['UpwindSlope'] = ls_slope
df['Significance'] = np.where((p_val['KS-P'].values <= 0.05) | (p_val['U-P'].values <= 0.05),'Sig','Non-Sig')

#print(df)
#df.to_csv('Modis/df_urban.csv', sep = ' ')

df_posDEI = df[df['DEI']>=0]
df_posDEI = df_posDEI[df_posDEI['City']!='Miami']
sig_slope = df_posDEI[df_posDEI['Significance']=='Sig']['UpwindSlope'].mean()
nonsig_slope = df_posDEI[df_posDEI['Significance']=='Non-Sig']['UpwindSlope'].mean()
sig_size = df_posDEI[df_posDEI['Significance']=='Sig']['UrbanSize'].mean()
nonsig_size = df_posDEI[df_posDEI['Significance']=='Non-Sig']['UrbanSize'].mean()

fig,ax = plt.subplots(figsize=(12,7))
ax.scatter(df_posDEI[df_posDEI['Significance']=='Non-Sig']['UrbanSize'],
           df_posDEI[df_posDEI['Significance']=='Non-Sig']['UpwindSlope'],
           s = [800*i for i in df_posDEI[df_posDEI['Significance']=='Non-Sig']['DEI'].values], alpha=0.7)
#ax.scatter(df_posDEI['UrbanSize'],df_posDEI['UpwindSlope'],s = [800*i for i in df_posDEI['DEI'].values])
ax.scatter(df_posDEI[df_posDEI['Significance']=='Sig']['UrbanSize'],
           df_posDEI[df_posDEI['Significance']=='Sig']['UpwindSlope'],c='r',
           s = [800*i for i in df_posDEI[df_posDEI['Significance']=='Sig']['DEI'].values], alpha=0.7)
texts = [ax.text(df_posDEI['UrbanSize'].values[i],df_posDEI['UpwindSlope'].values[i],
                 df_posDEI['City'].values[i],size=10) for i in range(len(df_posDEI['City'].values))]
adjust_text(texts)

ax.axhline(y=sig_slope, color='r',ls='-',label='Mean Imp Slope (Significant)')
ax.axhline(y=nonsig_slope, color='b',ls='-',label='Mean Imp Slope (Insignificant)')
ax.axvline(x=sig_size, color='r',ls='--',label='Mean Urban Size (Significant)')
ax.axvline(x=nonsig_size, color='b',ls='--',label='Mean Urban Size (Insignificant)')

ax.set_xlabel('Urban Size (km)',size = 16)
ax.set_ylabel('Upwind Imperviousness Slope ($km^{-1}$)',size = 16)
ax.legend(loc='upper right')

In [ ]:
## DEI vs. slope and DEI vs. urban size
plt.style.use(['science','no-latex'])

fig,ax = plt.subplots(1,2,figsize=(18,7),sharey=True)
ax[0].scatter(df_posDEI[df_posDEI['Significance']=='Non-Sig']['UrbanSize'],
           df_posDEI[df_posDEI['Significance']=='Non-Sig']['DEI'],alpha=0.7)

ax[0].scatter(df_posDEI[df_posDEI['Significance']=='Sig']['UrbanSize'],
           df_posDEI[df_posDEI['Significance']=='Sig']['DEI'],c='r',alpha=0.7)

ax[1].scatter(df_posDEI[df_posDEI['Significance']=='Non-Sig']['UpwindSlope'],
           df_posDEI[df_posDEI['Significance']=='Non-Sig']['DEI'],alpha=0.7)

ax[1].scatter(df_posDEI[df_posDEI['Significance']=='Sig']['UpwindSlope'],
           df_posDEI[df_posDEI['Significance']=='Sig']['DEI'],c='r',alpha=0.7)

texts = [ax[0].text(df_posDEI['UrbanSize'].values[i],df_posDEI['DEI'].values[i],
                 df_posDEI['City'].values[i],size=10) for i in range(len(df_posDEI['City'].values))]
adjust_text(texts)

texts = [ax[1].text(df_posDEI['UpwindSlope'].values[i],df_posDEI['DEI'].values[i],
                 df_posDEI['City'].values[i],size=10) for i in range(len(df_posDEI['City'].values))]
adjust_text(texts)

ax[0].set_xlabel('Urban Size (km)',size = 16)
ax[0].set_ylabel('DEI',size = 16)
ax[1].set_xlabel('Upwind Imperviousness Slope ($km^{-1}$)',size = 16)

In [ ]:
## city size based on imp fraction threshold
buf_dis1 = 60
buf_dis2 = 20
from adjustText import adjust_text
os.chdir('/glade/scratch/USERNAME')

plt.style.context(['science','no-latex'])

drct_px_60 = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis1)+'.csv',sep = ' ') # number of all pixels
drct_imp_60 = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis1)+'.csv',sep = ' ') # number of impervious pixels
drct_px_20 = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis2)+'.csv',sep = ' ') # number of all pixels
drct_imp_20 = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis2)+'.csv',sep = ' ') # number of impervious pixels

precip_t2 = pd.read_csv('./MRMS_PrecipData/T2Drct_SumPrecip'+str(buf_dis1)+'.csv',sep = ' ') # use 60km DEI
T2_drct_up = precip_t2['Top 2 drct'].to_list()

T2drct_px60_up = []
T2drct_imp60_up = []
T2drct_px20_up = []
T2drct_imp20_up = []
for i,j in enumerate(T2_drct_up):
    drcts = j.split('+')
    T2drct_px60_up.append(drct_px_60.iloc[i][drcts[0]]+drct_px_60.iloc[i][drcts[1]])
    T2drct_imp60_up.append(drct_imp_60.iloc[i][drcts[0]]+drct_imp_60.iloc[i][drcts[1]])
    T2drct_px20_up.append(drct_px_20.iloc[i][drcts[0]]+drct_px_20.iloc[i][drcts[1]])
    T2drct_imp20_up.append(drct_imp_20.iloc[i][drcts[0]]+drct_imp_20.iloc[i][drcts[1]])

T2drct_r2060_px = [i-j for i,j in zip(T2drct_px60_up,T2drct_px20_up)] ## number of total pixels in the ring between 20km and 60km
T2drct_r2060_imp = [i-j for i,j in zip(T2drct_imp60_up,T2drct_imp20_up)]

frac_imp2060_up = [i/j for i,j in zip(T2drct_r2060_imp,T2drct_r2060_px)]
frac_imp2060_up = [round(i,4) for i in frac_imp2060_up]
#print(frac_imp2060_up)
frac_imp60_up = [i/j for i,j in zip(T2drct_imp60_up,T2drct_px60_up)]
frac_imp60_up = [round(i,4) for i in frac_imp60_up]
#print(frac_imp60_up)

frac_imp20_up = [i/j for i,j in zip(T2drct_imp20_up,T2drct_px20_up)]
frac_imp20_up = [round(i,4) for i in frac_imp20_up]


gradient = [i/j for i,j in zip(frac_imp20_up,frac_imp60_up)]
gradient = [round(i,2) for i in gradient]


DEI_t2 = 2*(precip_t2['Downwind_Sum']-precip_t2['Upwind_Sum']) \
/(precip_t2['Downwind_Sum']+precip_t2['Upwind_Sum'])

####
p_val = pd.read_csv('./MRMS_PrecipData/T2Drct_SigLevel'+str(buf_dis1)+'.csv',sep = ' ') # use the 60km significance results
Sig_idx = p_val.index[(p_val['KS-P']<=0.05)|(p_val['U-P']<=0.05)].to_list()
gradient_filt = [gradient[i] for i in Sig_idx]
DEI_t2_filt = DEI_t2[Sig_idx].to_list()


x = np.arange(-60,60,3)
#df_imp_rate_t2['Atlanta'][df_imp_rate_t2['Atlanta']>=0.3].index.values
imp50_idx = []
for city in ls_city:
    imp50_idx.append(df_imp_rate_t2[city][df_imp_rate_t2[city]>=0.3].index.values) 
urb_size = []
for i in imp50_idx:
    urb_size.append(x[20]-x[i[0]])
urb_size_filt = [urb_size[i] for i in Sig_idx] # extract significant ones

# add one dimension of total precipitation at two prevailing directions
dat_precip = pd.read_csv('./MRMS_PrecipData/T2Drct_SumPrecip60.csv',sep=' ')
sum_precip = (dat_precip['Upwind_Sum']+dat_precip['Downwind_Sum'])/2/7 # annual top2 direction precip
sum_precip = sum_precip.to_list()
sum_precip_pos = [i for i,j in zip(sum_precip,DEI_t2) if j>=0]
sum_precip_filt = [sum_precip[i] for i in Sig_idx] # sig ones, including pos and neg
sum_precip_filt = [i for i,j in zip(sum_precip_filt,DEI_t2_filt) if j>=0] # pos only

urb_size = [i for i,j in zip(urb_size,DEI_t2) if j>=0] # get positive DEIs and related index
cities = [i for i,j in zip(ls_city,DEI_t2) if j>=0]
DEI_t2 = [j for j in DEI_t2 if j>=0]

urb_size_filt = [i for i,j in zip(urb_size_filt,DEI_t2_filt) if j>=0]
DEI_t2_filt = [j for j in DEI_t2_filt if j>=0]

print(np.corrcoef(urb_size_filt,DEI_t2_filt))
print(np.corrcoef(urb_size,DEI_t2))

print(urb_size_filt)
print(DEI_t2_filt)

fig,ax = plt.subplots(figsize=(12,8))
ax.scatter(urb_size,DEI_t2,s = [i for i in sum_precip_pos],alpha=0.5) #
ax.scatter(urb_size_filt,DEI_t2_filt,c='r',s = [i for i in sum_precip_filt],alpha=0.5)
texts = [ax.text(urb_size[i],DEI_t2[i],cities[i],size=10) for i in range(len(cities))]
adjust_text(texts)
ax.set_xlabel('Urban Size (km)',size = 16)
ax.set_ylabel('DEI',size = 16)
ax.set_xlim(right=40)
ax.set_title('Imp threshold: 0.3',size = 18)

### Test plotting

In [ ]:
buf_dis1 = 60
buf_dis2 = 20
from adjustText import adjust_text
os.chdir('/glade/scratch/USERNAME')

drct_px_60 = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis1)+'.csv',sep = ' ') # number of all pixels
drct_imp_60 = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis1)+'.csv',sep = ' ') # number of impervious pixels
drct_px_20 = pd.read_csv('./Modis/Results/Drct_all_px'+str(buf_dis2)+'.csv',sep = ' ') # number of all pixels
drct_imp_20 = pd.read_csv('./Modis/Results/Drct_imp_px'+str(buf_dis2)+'.csv',sep = ' ') # number of impervious pixels

precip_t2 = pd.read_csv('./MRMS_PrecipData/T2Drct_SumPrecip'+str(buf_dis2)+'.csv',sep = ' ') # use 20km precip
T2_drct_up = precip_t2['Top 2 drct'].to_list()

T2drct_px60_up = []
T2drct_imp60_up = []
T2drct_px20_up = []
T2drct_imp20_up = []
for i,j in enumerate(T2_drct_up):
    drcts = j.split('+')
    T2drct_px60_up.append(drct_px_60.iloc[i][drcts[0]]+drct_px_60.iloc[i][drcts[1]])
    T2drct_imp60_up.append(drct_imp_60.iloc[i][drcts[0]]+drct_imp_60.iloc[i][drcts[1]])
    T2drct_px20_up.append(drct_px_20.iloc[i][drcts[0]]+drct_px_20.iloc[i][drcts[1]])
    T2drct_imp20_up.append(drct_imp_20.iloc[i][drcts[0]]+drct_imp_20.iloc[i][drcts[1]])

T2drct_r2060_px = [i-j for i,j in zip(T2drct_px60_up,T2drct_px20_up)] ## number of total pixels in the ring between 20km and 60km
T2drct_r2060_imp = [i-j for i,j in zip(T2drct_imp60_up,T2drct_imp20_up)]

frac_imp2060_up = [i/j for i,j in zip(T2drct_r2060_imp,T2drct_r2060_px)]
frac_imp2060_up = [round(i,4) for i in frac_imp2060_up]
print(frac_imp2060_up)
frac_imp60_up = [i/j for i,j in zip(T2drct_imp60_up,T2drct_px60_up)]
frac_imp60_up = [round(i,4) for i in frac_imp60_up]
print(frac_imp60_up)

frac_imp20_up = [i/j for i,j in zip(T2drct_imp20_up,T2drct_px20_up)]
frac_imp20_up = [round(i,4) for i in frac_imp20_up]


gradient = [i/j for i,j in zip(frac_imp20_up,frac_imp60_up)]
gradient = [round(i,2) for i in gradient]


DEI_t2 = (precip_t2['Downwind_Sum']-precip_t2['Upwind_Sum']) \
/(precip_t2['Downwind_Sum']+precip_t2['Upwind_Sum'])

up20_norm = precip_t2['Upwind_Sum']/(precip_t2['Upwind_Sum']+precip_t2['Downwind_Sum'])
#up20_norm = up20_norm.drop([18])
#UD_ratio = precip_t2['Upwind_Sum']/precip_t2['Downwind_Sum']



city = precip_t2['City'].to_list()

#print(np.corrcoef(gradient_filt,UD_ratio_filt))

#up20_norm = up20_norm.drop([18]).to_list()
#del gradient[18]
#del city[18]

print(np.corrcoef(gradient,up20_norm))
plt.style.use('seaborn')
fig,ax = plt.subplots(figsize=(16,8))
ax.scatter(gradient,up20_norm) #
#ax.scatter(gradient_filt,UD_ratio_filt,color='red')
texts = [ax.text(gradient[i],up20_norm[i],city[i],size=10) for i in range(len(city))]
adjust_text(texts)
ax.set_xlabel('Upwind Impervious Ratio (20km/60km)',size=18)
#ax.set_ylabel('Downwind Enhancement Index (DEI)',size=18)
ax.set_ylabel('Normalized 20km Upwind Rainfall',size=18)
ax.set_title('Top2 Prevailing Directions',size = 20)